In [1]:
import numpy as np
import os
import yaml
from matplotlib import pyplot as plt
from scipy.signal import medfilt as medfilt
from scipy import signal
import csv


In [21]:
# converts a csv file to a python list of rows
def csvfile_to_list(fp):
    f = open(fp)
    reader = csv.reader(f)
    _ = next(reader)
    list_of_rows = [row for row in reader]
    return list_of_rows


# returns a list of filepaths collected from a parent directory and all subdirectories
def recursive_file_retrieval(parent_path, ignore_hidden_dirs=False, return_parent=True):
    
    file_path_list = []
    dir_list = []
    parent_paths = [parent_path]

    more_subdirs = True
    while more_subdirs == True:
        subdir_paths = []
        for i, parent_path in enumerate(parent_paths):
            # print(parent_path)
            if ignore_hidden_dirs:
                if os.path.basename(parent_path).startswith('.'):
                    continue

            dir_list.append(parent_path)
            r,dirs,files = next(os.walk(parent_path, topdown=True, onerror=None, followlinks=False)) 
            for f in files:
                file_path_list.append(os.path.join(r,f))

            # if there are more subdirectories
            if len(dirs) != 0:
                for d in dirs:
                    subdir_paths.append(os.path.join(r,d))

            # if we've finished going through subdirectories (each parent_path), stop that loop
            if i == len(parent_paths)-1:
                # if loop about to finish, change parent_paths content and restart loop
                if len(subdir_paths) != 0:
                    parent_paths = subdir_paths
                else:
                    more_subdirs = False
    
    if not return_parent: dir_list = dir_list[1:]

    return dir_list, file_path_list


def plot_ts(ts, hop_duration=0.016):
    time = np.arange(len(ts))*hop_duration

    # Plot the time series
    plt.plot(time, ts, label='Time Series')
    plt.xlabel('Time')
    plt.ylabel('Values')
    plt.title('Time Series Example')
    plt.legend()
    plt.grid(True)
    plt.show()
    plt.close()


def normalise(arr):
    mx = arr.max() # removed axis=1
    for i in range(mx.shape[0]):
        if mx[i] > 0:
            arr[i,:] /= mx[i]
    return arr


def tau_to_tempo(tau, hop=0.016, output='bpm'):
    period = tau * hop
    if output == 'bpm':
        return period_to_bpm(period)
    else:
        return period


def period_to_bpm(period):
    if period <= 0:
        period = 1e-12
    return 60/period


def beats_from_beatfile(fp):
    csv_rows = csvfile_to_list(fp)
    ts_beats = []
    for r in csv_rows:
        entry = r[0]
        entries = entry.split('\t')
        try:
            timestamp, beat = float(entries[0]), int(entries[1])
        except Exception as e:
            break
        ts_beats.append((timestamp, beat))
    return ts_beats


def temp_from_beatsfile(fp, output='bpm', summation='mean'):
    ts_beats = beats_from_beatfile(fp)
    timestamps = np.array([ts for ts, b in ts_beats])
    if summation == 'mean':
        mean_period = np.mean(np.diff(timestamps))
    elif summation == 'median':
        mean_period = np.median(np.diff(timestamps))
    if output == 'bpm':
        try:
            bpm = round(period_to_bpm(mean_period))
        except Exception as e:
            bpm = period_to_bpm(mean_period)
        return bpm 
    elif output == 'period':
        return mean_period
    

# get several onset arrays using several dsp approaches
def get_onset_arrs(specs):
   
    # get rms values
    rms_values = np.sqrt(np.mean(specs**2, axis=1))
    dim = specs.shape[1]
    # get high frequency content measure
    hfc_muliplier = np.asarray(range(dim))+1
    hfc = np.mean((specs * hfc_muliplier), axis=1)
    # get spectral flux
    delta_spectrogram = np.diff(specs, axis=0)
    delta_spectrogram = np.concatenate((np.zeros((1,dim)), delta_spectrogram))
    mask = delta_spectrogram > 0
    delta_spec_up = delta_spectrogram * mask
    spec_flux = np.sum(delta_spec_up, axis=1)

    return rms_values, hfc, spec_flux
    

# Clean up onset arrays: Smoothen, normalise, denoise, peak-isolation
def cleanup_onset_array(ts, mf_k=3, threshold=0.5, autocor_peak_dist=5):
    med_ts = ts - medfilt(ts, mf_k)
    med_grad_ts = np.gradient(med_ts)
    stdise_med_grad_ts = (med_grad_ts - np.mean(med_grad_ts)) / np.std(med_grad_ts)
    min_max_med_grad_ts = med_grad_ts / np.max(med_grad_ts)
    normmed_med_grad_ts = stdise_med_grad_ts
    thresholded_ts = normmed_med_grad_ts.copy()
    thresholded_ts[thresholded_ts < threshold] = 0
    onset_peak_indices, peak_value_dict = signal.find_peaks(thresholded_ts, height=0, distance=autocor_peak_dist)
    peak_values = peak_value_dict['peak_heights']
    trimmed_odf = np.zeros_like(thresholded_ts)
    trimmed_odf[onset_peak_indices] = peak_values
    return trimmed_odf


# get tempo candidantes from onset array
def get_tempo(onsets, tau_limits):
    autocor = np.correlate(onsets, onsets, mode='full')[int(len(np.correlate(onsets, onsets, mode='full'))//2)+1:]
    acf_peak_indices, acf_peak_value_dict = signal.find_peaks(autocor, height=np.max(autocor)/2, distance=5)
    acf_peak_values = acf_peak_value_dict['peak_heights']
    peaked_autocor = np.zeros_like(autocor)
    peaked_autocor[acf_peak_indices] = acf_peak_values
    realistic_autocor_cands = peaked_autocor[tau_limits[1]:tau_limits[0]]
    best_tau_samples = np.argpartition(realistic_autocor_cands, -4)[-4:]
    return [round(tau_to_tempo(t)) for t in best_tau_samples]


def beatsfp_from_featsfp(feats_fp, beats_dir):
    fn = os.path.basename(feats_fp)
    genre, id = fn.split('.')[:2]
    beats_basename = 'gtzan_' +genre +'_' + id +'.beats'
    beats_fp = os.path.join(beats_dir, beats_basename)
    return beats_fp

In [3]:
feats_dir = '/Users/brendanoconnor/Desktop/career/tempoEstimationTask/stored_feats'
beats_dir = '/Users/brendanoconnor/Downloads/MLExcerciseBrendan/data/GTZAN/beats'
threshold = 0.5
bpm_limits = [50, 250]
mf_k=3
ofd_idx = 1

In [5]:
with open(os.path.join(feats_dir, "feat_params.yaml"), "rb") as handle:
    feat_params = yaml.load(handle, Loader=yaml.FullLoader)

print(feat_params)

# onset detection function generators
hop_duration = feat_params['hop_size'] / feat_params['sr']
dim_size = feat_params['num_mels']
tau_limits = [round(1 / (bpm / 60 * hop_duration)) for bpm in bpm_limits]

{'fft_size': 1024, 'fmax': 7600, 'fmin': 90, 'hop_size': 256, 'num_mels': 80, 'sr': 16000}


### Collect premade features and derive onsets from them in several ways

In [6]:
_, fps = recursive_file_retrieval(feats_dir)
filt_fps = [fp for fp in fps if not fp.startswith('.') and fp.endswith('npy')]

fps_onsets = []
feats_fp = '/Users/brendanoconnor/Desktop/career/tempoEstimationTask/stored_feats/pop/pop.00000.wav.npy'

for feats_fp in filt_fps:
    feats = np.load(feats_fp)
    fps_onsets.append((feats_fp, get_onset_arrs(feats)))



In [22]:

for fp_onset in fps_onsets:
    fp, onsets = fp_onset[0], fp_onset[1]
    onset_arr = onsets[ofd_idx]
    cleanedup_onset_arr = cleanup_onset_array(onset_arr)
    bpms = get_tempo(cleanedup_onset_arr, tau_limits)
    beats_fp = beatsfp_from_featsfp(fp, beats_dir)
    if not os.path.exists(beats_fp):
        continue
    print(temp_from_beatsfile(beats_fp, summation='mean'), bpms)

83 [208, 536, 72, 129]
85 [75, 114, 417, 87]
59 [221, 179, 188, 938]
108 [179, 69, 188, 197]
80 [87, 69, 197, 469]
93 [234, 188, 156, 85]
99 [179, 89, 101, 80]
93 [179, 208, 938, 85]
63 [85, 341, 65, 129]
72 [179, 104, 375, 64]
110 [1875, 375, 114, 96]
93 [221, 85, 938, 150]
126 [188, 64, 197, 65]
110 [3750, 208, 72, 417]
67 [64, 69, 312, 144]
116 [117, 234, 60000000000000, 77]
101 [125, 94, 65, 179]
80 [197, 69, 536, 121]
104 [221, 3750, 188, 67]
106 [68, 197, 1875, 101]
79 [188, 469, 68, 121]
82 [71, 536, 60000000000000, 125]
115 [221, 114, 77, 60000000000000]
128 [87, 625, 179, 134]
113 [179, 75, 110, 3750]
115 [77, 60000000000000, 221, 114]
89 [104, 80, 144, 750]
113 [179, 75, 110, 3750]
57 [64, 188, 75, 234]
131 [197, 68, 750, 78]
125 [134, 268, 87, 60000000000000]


98 [234, 188, 75, 64]
84 [234, 208, 188, 64]
97 [197, 288, 89, 170]
128 [139, 66, 89, 268]
59 [121, 234, 80, 60000000000000]
96 [188, 938, 87, 163]
94 [64, 188, 156, 87]
100 [75, 1250, 170, 312]
61 [938, 129, 82, 250]
111 [107, 3750, 208, 72]
55 [3750, 107, 208, 72]
81 [536, 91, 197, 71]
101 [179, 1250, 94, 64]
100 [121, 1250, 94, 64]
102 [150, 125, 96, 179]
110 [107, 3750, 208, 72]
64 [3750, 163, 69, 288]
103 [1875, 341, 78, 66]
88 [139, 78, 750, 234]
115 [188, 197, 221, 77]
120 [197, 208, 250, 121]
134 [69, 144, 312, 94]
102 [221, 197, 179, 125]
105 [469, 67, 1875, 188]
70 [188, 74, 156, 341]
119 [121, 234, 80, 60000000000000]
88 [101, 750, 78, 234]
113 [208, 107, 60000000000000, 74]
113 [87, 144, 65, 74]
132 [170, 1250, 91, 312]
109 [197, 104, 71, 3750]
91 [221, 750, 150, 82]
127 [625, 65, 134, 87]
105 [67, 80, 1875, 129]
54 [197, 64, 104, 71]
106 [234, 208, 188, 197]
64 [288, 89, 134, 75]
65 [68, 89, 288, 134]
98 [197, 288, 89, 170]
124 [197, 144, 268, 129]
99 [75, 91, 170, 1250]
7

/Users/brendanoconnor/Desktop/career/tempoEstimationTask/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/brendanoconnor/Desktop/career/tempoEstimationTask/venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


73 [129, 94, 536, 77]
124 [85, 469, 268, 129]
115 [221, 114, 60000000000000, 77]
120 [107, 250, 82, 125]
108 [72, 3750, 82, 197]
52 [288, 188, 82, 101]
74 [288, 77, 536, 94]
100 [66, 134, 72, 221]
71 [96, 64, 82, 417]
188 [188, 85, 156, 938]
153 [85, 66, 114, 179]
134 [77, 94, 312, 150]
197 [375, 91, 188, 1250]
103 [78, 69, 1250, 341]
53 [129, 163, 101, 68]
74 [65, 80, 234, 139]
95 [129, 156, 83, 74]
108 [3750, 208, 83, 104]
161 [71, 144, 121, 469]
155 [66, 60000000000000, 117, 469]
104 [67, 188, 938, 91]
166 [94, 221, 625, 134]
133 [188, 64, 94, 312]
56 [60000000000000, 75, 288, 129]
158 [188, 69, 121, 536]
107 [69, 3750, 134, 375]
215 [69, 1875, 197, 101]
208 [67, 99, 3750, 188]
195 [188, 179, 1250, 85]
124 [1250, 268, 170, 85]
136 [188, 64, 96, 312]
55 [85, 96, 170, 3750]
62 [66, 536, 197, 3750]
149 [375, 250, 114, 64]
242 [72, 64, 375, 104]
184 [74, 536, 221, 129]
129 [77, 312, 107, 87]
43 [234, 938, 139, 104]
178 [110, 144, 80, 750]
209 [94, 75, 1250, 179]
83 [60000000000000, 208,